# Holography Tools prototype

Demonstrates the prototype holo tools GUI panels

This currently relies on the branch https://github.com/sk1p/LiberTEM-holo/tree/core-reconstr-changes, uncomment to install:

In [ ]:
# !pip -q install -q git+https://github.com/sk1p/LiberTEM-holo.git@core-reconstr-changes

In [ ]:
import numpy as np
import panel as pn
import libertem.api as lt

In [ ]:
from libertem_ui.ui_context import UIContext
from libertem_ui.live_plot import ApertureFigure
from libertem_ui.applications.holography.panels import ApertureBuilder, ApertureConfig

## Data

Build a fake holo stack from some `skimage` datasets

In [ ]:
from libertem_holo.base.generate import hologram_frame
from skimage.data import cat, gravel, astronaut
from skimage.transform import resize

In [ ]:
shape = (512, 512)
angle = np.random.uniform(0, 360)
sampling = np.random.uniform(3, 5)
frames = []
for c in range(4):
    amp = resize(cat()[..., 0] / 255, shape)
    phase = resize(gravel()[:64, :64] / 255, amp.shape, order=2) * 2 * np.pi - np.pi
    for _ in range(c):
        phase = np.rot90(phase)
    frames.append(hologram_frame(amp, phase, visibility=0.2, sampling=sampling, f_angle=angle))
frames = np.asarray(frames).astype(np.float32)

Display the generated wave amplituce + phase:

In [ ]:
left = ApertureFigure.new(amp, title="amp")
right = ApertureFigure.new(phase, title="phase")
pn.Row(left.layout, right.layout)

## Context + Dataset + Window

At the moment this is only running on a memory dataset, but better LT integration (and abstraction) would be standard).

In [ ]:
ctx = lt.Context.make_with('inline')
ds = ctx.load('memory', data=frames)

The state of the parameters can be extracted and loaded into the window via a `NamedTuple`:

In [ ]:
state = ApertureConfig(
    sb_pos=(80, 20),
    radius=8,
    window_size=(32, 32)
)

Passing the `state` is optional, if not provided then a default state is used.

Construct and display the window:

In [ ]:
window = ApertureBuilder.using(ctx, ds, state=state)
window.layout()

Extract the current state of the parameters from the window:

In [ ]:
window.state